In [1]:
import pandas
import scipy
from docplex.mp.model import Model
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy

In [2]:
COLUMNAS = ["a", "b", "c", "d", "e", "f", "g", "h"]
FILAS = [1, 2, 3, 4, 5, 6, 7, 8]
CASILLAS = [f"{columna}{fila}" for columna in COLUMNAS for fila in FILAS]
PIEZAS = ["K", "R", "Q", "N", "B"]

In [3]:
def ataque_torre(casilla: str) -> list[str]:
    columna_torre = COLUMNAS.index(casilla[0]) + 1
    fila_torre = int(casilla[1])
    casillas_atacadas = []
    for fila in range(1, 9):
        if fila != fila_torre:
            casillas_atacadas.append(f"{COLUMNAS[columna_torre - 1]}{fila}")
    for i, columna in enumerate(COLUMNAS):
        if i + 1 != columna_torre:
            casillas_atacadas.append(f"{columna}{fila_torre}")
    return casillas_atacadas


def ataque_alfil(casilla: str) -> list[str]:
    columna_alfil = COLUMNAS.index(casilla[0])
    fila_alfil = int(casilla[1])
    casillas_atacadas = []
    direcciones = [(-1, -1), (-1, 1), (1, -1), (1, 1)]
    for dx, dy in direcciones:
        x, y = columna_alfil, fila_alfil
        while 0 <= x + dx < 8 and 0 <= y + dy <= 8:
            x += dx
            y += dy
            if 0 <= x < 8 and 1 <= y <= 8:
                casillas_atacadas.append(f"{COLUMNAS[x]}{y}")
    return casillas_atacadas


def ataque_dama(casilla: str) -> list[str]:
    columna_dama = COLUMNAS.index(casilla[0])
    fila_dama = int(casilla[1])
    casillas_atacadas = []
    for i in range(8):
        if i != columna_dama:
            casillas_atacadas.append(f"{COLUMNAS[i]}{fila_dama}")
        if i + 1 != fila_dama:
            casillas_atacadas.append(f"{COLUMNAS[columna_dama]}{i + 1}")

    direcciones = [(-1, -1), (-1, 1), (1, -1), (1, 1)]
    for dx, dy in direcciones:
        x, y = columna_dama, fila_dama
        while 0 <= x + dx < 8 and 0 <= y + dy <= 8:
            x += dx
            y += dy
            if 0 <= x < 8 and 1 <= y <= 8:
                casillas_atacadas.append(f"{COLUMNAS[x]}{y}")
    return list(set(casillas_atacadas))


def ataque_rey(casilla: str) -> list[str]:
    columna_rey = COLUMNAS.index(casilla[0])
    fila_rey = int(casilla[1])
    movimientos = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    casillas_atacadas = []
    for dx, dy in movimientos:
        nueva_columna = columna_rey + dx
        nueva_fila = fila_rey + dy
        if 0 <= nueva_columna < 8 and 1 <= nueva_fila <= 8:
            casilla_atacada = f"{COLUMNAS[nueva_columna]}{nueva_fila}"
            casillas_atacadas.append(casilla_atacada)
    return casillas_atacadas


def ataque_caballo(casilla: str) -> list[str]:
    columna_caballo = COLUMNAS.index(casilla[0])
    fila_caballo = int(casilla[1])
    movimientos = [(-2, -1), (-2, 1), (-1, -2), (1, -2), (-1, 2), (1, 2), (2, -1), (2, 1)]
    casillas_atacadas = []
    for dx, dy in movimientos:
        nueva_columna = columna_caballo + dx
        nueva_fila = fila_caballo + dy
        if 0 <= nueva_columna < 8 and 1 <= nueva_fila <= 8:
            casilla_atacada = f"{COLUMNAS[nueva_columna]}{nueva_fila}"
            casillas_atacadas.append(casilla_atacada)
    return casillas_atacadas

In [4]:
ATAQUE = {"K": ataque_rey, "R": ataque_torre, "Q": ataque_dama, "N": ataque_caballo, "B": ataque_alfil}
GANANCIA_PIEZA = {"K": 14, "R": 28, "Q": 28, "N": 7, "B": 16}

In [5]:
model = Model("max_chess")

In [6]:
X = model.binary_var_matrix(PIEZAS, CASILLAS, name="X")

In [7]:
model.maximize(model.sum(GANANCIA_PIEZA[pieza] * X[pieza, casilla] for pieza in PIEZAS for casilla in CASILLAS))

In [8]:
for pieza in PIEZAS:
    model.add_constraint(model.sum(X[pieza, casilla] for casilla in CASILLAS) >= 1, f"Place_{pieza}")

for casilla in CASILLAS:
    model.add_constraint(model.sum(X[pieza, casilla] for pieza in PIEZAS) <= 1, f"Spot_{casilla}")

for pieza in PIEZAS:
    for casilla in CASILLAS:
        CASILLAS_ATACADAS_PIEZAS = ATAQUE[pieza](casilla)
        for casilla_atacada in CASILLAS_ATACADAS_PIEZAS:
            for pieza_ in PIEZAS:
                model.add_constraint(X[pieza_, casilla_atacada] <= 1 - X[pieza, casilla], f"Attack_{pieza}_{casilla}_{pieza_}_{casilla_atacada}")

In [9]:
model.set_time_limit(60)

In [10]:
solution = model.solve(log_output=True)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               60
Found incumbent of value 93.000000 after 0.00 sec. (0.61 ticks)


Tried aggregator 1 time.
MIP Presolve eliminated 17427 rows and 0 columns.
MIP Presolve modified 977 coefficients.
Reduced MIP has 982 rows, 320 columns, and 13750 nonzeros.
Reduced MIP has 320 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.12 sec. (53.14 ticks)
Probing time = 0.02 sec. (0.55 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 982 rows, 320 columns, and 13750 nonzeros.
Reduced MIP has 320 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (14.04 ticks)
Probing time = 0.00 sec. (0.55 ticks)
Clique table members: 977.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time = 0.33 sec. (173.85 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                           93.0000     5952.0000 

In [11]:
TABLERO = {casilla: pieza for pieza in PIEZAS for casilla in CASILLAS if X[pieza, casilla].solution_value > 0.9}

In [12]:
PIEZAS_ICONOS = {"K": "♚", "R": "♜", "Q": "♛", "N": "♞", "B": "♝"}

In [13]:
solution.objective_value

281.0

In [14]:
# Crear el tablero de ajedrez
tablero = numpy.zeros((8, 8), dtype=int)
tablero[1::2, ::2] = 1
tablero[::2, 1::2] = 1


# Crear la figura del tablero usando Heatmap
fig = go.Figure()

fig.add_trace(
    go.Heatmap(
        z=tablero,
        colorscale=["#eeeed2", "#769656"],
        showscale=False,
        x=COLUMNAS,
        y=list(map(str, FILAS)),
        hoverinfo="none",
    )
)

for casilla, pieza in TABLERO.items():
    x = COLUMNAS.index(casilla[0])
    y = 7 - FILAS.index(int(casilla[1]))
    fig.add_annotation(x=x, y=y, text=PIEZAS_ICONOS[pieza], showarrow=False, font_size=50, font_color="black")


fig.update_layout(coloraxis_showscale=False)
fig.update_layout(height=800, width=800, title_text=f"Utilidad: {solution.objective_value}", template="ggplot2")
fig.show()